In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
device='cuda:0' if torch.cuda.is_available() else 'cpu'

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

Divide the training set and the test set

In [ ]:
df_suzhou=pd.read_excel("./data/P1.xlsx")
df_suzhou['日期'] = pd.to_datetime(df_suzhou['日期'])
X1_suzhou=df_suzhou.drop(columns=['平均负荷','碳排放强度','排汽温度', '日期','机组'])
y1_suzhou=df_suzhou['碳排放强度']

df_maanshan=pd.read_excel("./data/P3.xlsx")
df_maanshan['日期'] = pd.to_datetime(df_maanshan['日期'])
X1 = df_maanshan.drop(columns=['平均负荷','碳排放强度','排汽温度', '日期'])
y1 = df_maanshan['碳排放强度']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

1.SVM

In [ ]:
svr_model = SVR(gamma=0.005,C=1000)
svr_model.fit(X1_suzhou, y1_suzhou)
svm_y_pred_AonB = svr_model.predict(X1_train)
svm_y_pred_AonB_as_test = svr_model.predict(X1_test)
XB_with_graft=np.hstack((X1_train,pd.DataFrame({'AonB':svm_y_pred_AonB})))
XB_with_graft_as_test=np.hstack((X1_test,pd.DataFrame({'AonB':svm_y_pred_AonB_as_test})))
svr_model.fit(XB_with_graft,y1_train)
svm_y_pred = svr_model.predict(XB_with_graft_as_test)
svm_mse = mean_squared_error(y1_test, svm_y_pred)
svm_rmse = np.sqrt(svm_mse)
svm_mae = mean_absolute_error(y1_test, svm_y_pred)
svm_mape = mean_absolute_percentage_error(y1_test, svm_y_pred)
svm_r2 = r2_score(y1_test, svm_y_pred)

print(f"svm_MSE: {svm_mse:.6f}")
print(f"svm_RMSE: {svm_rmse:.6f}")
print(f"svm_MAE: {svm_mae:.6f}")
print(f"svm_MAPE: {svm_mape*100:.6f}")
print(f"svm_R Square: {svm_r2:.6f}")

In [ ]:
plt.figure(figsize=(18, 9))
plt.plot(svm_y_pred, label='预测值')
plt.plot(y1_test.values, label='实际值')

plt.legend(fontsize=15)
plt.xlabel("样本序列",fontsize = 15,fontweight='bold')
plt.ylabel("碳排放强度/(g/kWh)",fontsize = 15,fontweight='bold')

plt.tight_layout()
plt.show()

2.XGB

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # random_state=42
)

xgb_model.fit(X1_suzhou, y1_suzhou)
xgb_y_pred_AonB = xgb_model.predict(X1_train)
xgb_y_pred_AonB_as_test = xgb_model.predict(X1_test)
XB_with_graft=np.hstack((X1_train,pd.DataFrame({'AonB':xgb_y_pred_AonB})))
XB_with_graft_as_test=np.hstack((X1_test,pd.DataFrame({'AonB':xgb_y_pred_AonB_as_test})))
xgb_model.fit(XB_with_graft,y1_train)
xgb_y_pred = xgb_model.predict(XB_with_graft_as_test)
xgb_mse = mean_squared_error(y1_test, xgb_y_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_mae = mean_absolute_error(y1_test, xgb_y_pred)
xgb_mape = mean_absolute_percentage_error(y1_test, xgb_y_pred)
xgb_r2 = r2_score(y1_test, xgb_y_pred)

print(f"xgb_MSE: {xgb_mse:.6f}")
print(f"xgb_RMSE: {xgb_rmse:.6f}")
print(f"xgb_MAE: {xgb_mae:.6f}")
print(f"xgb_MAPE: {xgb_mape*100:.6f}")
print(f"xgb_R Square: {xgb_r2:.6f}")

In [ ]:
import matplotlib.lines as mlines
plt.figure(figsize=(18, 9))
plt.plot(xgb_y_pred, label='预测值', linewidth=1,color = "#4874CB")
plt.plot(y1_test.values, label='实际值', linewidth=1, color='#CA7373')
plt.scatter(range(len(xgb_y_pred)), xgb_y_pred, label='预测值',  alpha=0.7,marker= "X",color = "#4874CB")
plt.scatter(range(len(y1_test.values)), y1_test.values, label='实际值', alpha=0.7, color='#CA7373')
line1 = mlines.Line2D([], [], color="#4874CB", marker="X", markersize=6, label="预测值")
line2 = mlines.Line2D([], [], color="#CA7373", marker="o", markersize=6, label="实际值")

plt.legend(handles=[line1, line2], fontsize=12.5)
plt.xlabel("样本序列",fontsize = 15,fontweight='bold')
plt.ylabel("碳排放强度/(g/kWh)",fontsize = 15,fontweight='bold')

plt.tight_layout()
plt.show()

3.RF

In [ ]:
correlation_matrix = df_maanshan.drop(columns=['排汽温度', '日期']).corr()
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_jobs=28,max_depth=7)
# rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X1_suzhou, y1_suzhou)
rf_y_pred_AonB = rf_model.predict(X1_train)
rf_y_pred_AonB_as_test = rf_model.predict(X1_test)
XB_with_graft=np.hstack((X1_train,pd.DataFrame({'AonB':rf_y_pred_AonB})))
XB_with_graft_as_test=np.hstack((X1_test,pd.DataFrame({'AonB':rf_y_pred_AonB_as_test})))
rf_model.fit(XB_with_graft,y1_train)
rf_y_pred = rf_model.predict(XB_with_graft_as_test)
rf_mse = mean_squared_error(y1_test, rf_y_pred)
rf_rmse = np.sqrt(rf_mse)
rf_mae = mean_absolute_error(y1_test, rf_y_pred)
rf_mape = mean_absolute_percentage_error(y1_test, rf_y_pred)
rf_r2 = r2_score(y1_test, rf_y_pred)

print(f"rf_MSE: {rf_mse:.6f}")
print(f"rf_RMSE: {rf_rmse:.6f}")
print(f"rf_MAE: {rf_mae:.6f}")
print(f"rf_MAPE: {rf_mape*100:.6f}")
print(f"rf_R Square: {rf_r2:.6f}")

In [ ]:
plt.figure(figsize=(18, 9))
plt.plot(rf_y_pred, label='预测值')
plt.plot(y1_test.values, label='实际值')

plt.legend(fontsize=15)
plt.xlabel("样本序列",fontsize = 15,fontweight='bold')
plt.ylabel("碳排放强度/(g/kWh)",fontsize = 15,fontweight='bold')

plt.tight_layout()
plt.show()

4.LR

In [ ]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X1_suzhou, y1_suzhou)
lr_y_pred_AonB = lr_model.predict(X1_train)
lr_y_pred_AonB_as_test = lr_model.predict(X1_test)
XB_with_graft=np.hstack((X1_train,pd.DataFrame({'AonB':lr_y_pred_AonB})))
XB_with_graft_as_test=np.hstack((X1_test,pd.DataFrame({'AonB':lr_y_pred_AonB_as_test})))
lr_model.fit(XB_with_graft,y1_train)
lr_y_pred = lr_model.predict(XB_with_graft_as_test)
lr_mse = mean_squared_error(y1_test, lr_y_pred)
lr_rmse = np.sqrt(lr_mse)
lr_mae = mean_absolute_error(y1_test, lr_y_pred)
lr_mape = mean_absolute_percentage_error(y1_test, lr_y_pred)
lr_r2 = r2_score(y1_test, lr_y_pred)

print(f"lr_MSE: {lr_mse:.6f}")
print(f"lr_RMSE: {lr_rmse:.6f}")
print(f"lr_MAE: {lr_mae:.6f}")
print(f"lr_MAPE: {lr_mape*100:.6f}")
print(f"lr_R Square: {lr_r2:.6f}")

In [ ]:
plt.figure(figsize=(18, 9))
plt.plot(lr_y_pred, label='预测值')
plt.plot(y1_test.values, label='实际值')

plt.legend(fontsize=15)
plt.xlabel("样本序列",fontsize = 15,fontweight='bold')
plt.ylabel("碳排放强度/(g/kWh)",fontsize = 15,fontweight='bold')

plt.tight_layout()
plt.show()

5.DNN

In [ ]:
device='cuda:0' if torch.cuda.is_available() else 'cpu'
set_seed(42)
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
scaler3 = MinMaxScaler()
X1_suzhou_scaled=scaler1.fit_transform(X1_suzhou)
y1_suzhou_scaled=scaler1.fit_transform(y1_suzhou.to_frame())
X1_train_scaled=scaler2.fit_transform(X1_train)
y1_train_scaled=scaler2.fit_transform(y1_train.to_frame())
X1_test_scaled=scaler3.fit_transform(X1_test)
y1_test_scaled=scaler3.fit_transform(y1_test.to_frame())
X1_suzhou_tensor=torch.FloatTensor(X1_suzhou_scaled).to(device)
y1_suzhou_tensor=torch.FloatTensor(y1_suzhou_scaled).view(-1,1).to(device)
X1_train_tensor=torch.FloatTensor(X1_train_scaled).to(device)
y1_train_tensor=torch.FloatTensor(y1_train_scaled).view(-1,1).to(device)
X1_test_tensor=torch.FloatTensor(X1_test_scaled).to(device)
y1_test_tensor=torch.FloatTensor(y1_test_scaled).view(-1,1).to(device)

class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # 第一个隐藏层
        self.fc2 = nn.Linear(64, 32)          # 第二个隐藏层
        self.fc3 = nn.Linear(32, 1)           # 输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_size = X1_suzhou.shape[1]
model_a = SimpleNN(input_size).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model_a.parameters(), lr=0.17)
num_epochs = 5000
for epoch in range(num_epochs):
    model_a.train()
    optimizer.zero_grad()
    outputs = model_a(X1_suzhou_tensor)
    loss = criterion(outputs, y1_suzhou_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 200 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

model_a.eval()
with torch.no_grad():
    dnn_y_pred_AonB = model_a(X1_train_tensor)
    dnn_y_pred_AonB_as_test = model_a(X1_test_tensor)
XB_with_graft=np.hstack((X1_train_scaled,pd.DataFrame({'AonB':dnn_y_pred_AonB.cpu().numpy().flatten()})))
XB_with_graft_as_test=np.hstack((X1_test_scaled,pd.DataFrame({'AonB':dnn_y_pred_AonB_as_test.cpu().numpy().flatten()})))
XB_with_graft_tensor=torch.FloatTensor(XB_with_graft).to(device)
XB_with_graft_as_test_tensor=torch.FloatTensor(XB_with_graft_as_test).to(device)
input_size_b = XB_with_graft.shape[1]
model_b = SimpleNN(input_size_b).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model_b.parameters(), lr=0.05)
num_epochs = 2000
for epoch in range(num_epochs):
    model_b.train()
    optimizer.zero_grad()
    outputs = model_b(XB_with_graft_tensor)
    loss = criterion(outputs, y1_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 200 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

model_b.eval()
with torch.no_grad():
    dnn_y_pred_B = model_b(XB_with_graft_as_test_tensor)

dnn_y_pred_B=dnn_y_pred_B.cpu().numpy().flatten()
dnn_y_pred_B=scaler3.inverse_transform(pd.DataFrame({'pred':dnn_y_pred_B}))

dnn_mse = mean_squared_error(y1_test, dnn_y_pred_B)
dnn_rmse = np.sqrt(dnn_mse)
dnn_mae = mean_absolute_error(y1_test, dnn_y_pred_B)
dnn_mape = mean_absolute_percentage_error(y1_test, dnn_y_pred_B)
dnn_r2 = r2_score(y1_test, dnn_y_pred_B)

print(f"dnn_MSE: {dnn_mse:.6f}")
print(f"dnn_RMSE: {dnn_rmse:.6f}")
print(f"dnn_MAE: {dnn_mae:.6f}")
print(f"dnn_MAPE: {dnn_mape*100:.6f}")
print(f"dnn_R Square: {dnn_r2:.6f}")

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(18, 9))
plt.plot(dnn_y_pred_B, label='预测值')
plt.plot(y1_test.values, label='实际值')

plt.legend(fontsize=15)
plt.xlabel("样本序列",fontsize = 15,fontweight='bold')
plt.ylabel("碳排放强度/(g/kWh)",fontsize = 15,fontweight='bold')

plt.tight_layout()
plt.show()